In [ ]:
# Import libraries
from scipy.cluster.hierarchy import dendrogram, linkage
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import nibabel as nib
from sklearn.svm import SVR
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import ElasticNetCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import r2_score, roc_curve, auc
import SimpleITK as sitk
import six
from radiomics import featureextractor 
import numpy as np
import os
import pickle
import pandas as pd
import logging
from scipy.stats import linregress
from sklearn.linear_model import QuantileRegressor
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import r_regression
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import RANSACRegressor
import os
import sys
sys.path.append('../')
import matplotlib.pyplot as plt
from pylab import rcParams
import pandas as pd
import warnings
from sklearnex import patch_sklearn, config_context
from sklearn.cluster import DBSCAN
from sklearn.exceptions import ConvergenceWarning
from sklearn import preprocessing as skp
from sklearn import model_selection as sms
from sklearn import feature_selection as skf
from sklearn import linear_model as slm
import numpy as np
import scipy.stats as stats
from IPython.display import HTML
import util as util
import nibabel as nib
import os
import pickle
from torch import nn
patch_sklearn()

In [ ]:
plt.style.use('dark_background')
plt.rcParams["figure.figsize"] = (15,5)
def remove_keymap_conflicts(new_keys_set):
    for prop in plt.rcParams:
        if prop.startswith('keymap.'):
            keys = plt.rcParams[prop]
            remove_list = set(keys) & new_keys_set
            for key in remove_list:
                keys.remove(key)

def multi_slice_viewer(volume):
    remove_keymap_conflicts({'j', 'k'})
    fig, ax = plt.subplots()
    ax.volume = volume
    ax.index = volume.shape[0]//2
    ax.imshow(volume[ax.index])
    fig.canvas.mpl_connect('key_press_event', process_key)

def process_key(event):
    fig = event.canvas.figure
    ax = fig.axes[0]
    if event.key == 'j':
        previous_slice(ax)
    elif event.key == 'k':
        next_slice(ax)
    fig.canvas.draw()
    

def previous_slice(ax):
    volume = ax.volume
    ax.index = (ax.index-1) % volume.shape[0] 
    ax.images[0].set_array(volume[ax.index])

def next_slice(ax):
    volume = ax.volume
    ax.index = (ax.index+1) % volume.shape[0]
    ax.images[0].set_array(volume[ax.index])

In [ ]:
# Set window level
level = 0
window = 500
m1=level-window/2
m2=level+window/2
visualize = 1
# Load data
nrows = 256
ncols = 256
nslices = 160
segs = []
qsms = []
laros = []
voxel_sizes = []
trackers = []
q_directory = '/home/ali/RadDBS-QSM/data/nii/chh/orig/qsm/'
q_directory = os.listdir(q_directory)
q_directory = sorted(q_directory)
qu_directory = '/home/ali/RadDBS-QSM/data/nii/chh/lr_roi_ft'
qu_directory = os.listdir(qu_directory)
qu_directory = sorted(qu_directory)
s_directory = '/home/ali/RadDBS-QSM/data/nii/chh/orig/seg/'
s_directory = os.listdir(s_directory)
s_directory = sorted(s_directory)
m_directory = '/home/ali/RadDBS-QSM/data/nii/chh/masks'
m_directory = os.listdir(m_directory)
m_directory = sorted(m_directory)
case_list = []
d_count = 0
if visualize == 1:
    for filename in q_directory:
    
        seg_filename = s_directory[d_count]
        laro_filename = qu_directory[d_count]
        mask_filename = m_directory[d_count]
        seg = nib.load('/home/ali/RadDBS-QSM/data/nii/chh/orig/seg/'+seg_filename)
        mask = nib.load('/home/ali/RadDBS-QSM/data/nii/chh/masks/'+mask_filename)
        voxel_size = seg.header['pixdim'][0:3]
        voxel_sizes.append(voxel_size)
        segs.append(seg.get_fdata()[:nrows,:ncols,:nslices])
        qsm = nib.load('/home/ali/RadDBS-QSM/data/nii/chh/orig/qsm/'+filename)
        qsms.append(qsm.get_fdata()[:nrows,:ncols,:nslices])

        laro = nib.load('/home/ali/RadDBS-QSM/data/nii/chh/lr_roi_ft/'+laro_filename)
        laros.append(1000*laro.get_fdata()[:nrows,:ncols,:nslices])
        print('Appending arrays with segmentation',seg_filename,'QSM,',filename,
              'LARO,',laro_filename,'and mask',mask_filename)
        case_list.append(filename)
        n_cases = len(segs)
        d_count = d_count+1




In [ ]:
df = pd.read_csv('/home/ali/RadDBS-QSM/data/xlxs/updrs_iii_chh.csv')
df

In [ ]:
# Patient IDs
subject_id = np.asarray(df[df.columns[0]])[1:]

# Only extract ROI if it is present in all cases
seg_labels_all = segs[0]
case_number = np.zeros_like(np.asarray(s_directory))
for i in range(n_cases):
    case_number[i] = float(s_directory[i][:2])
subject_id_corr = subject_id[np.in1d(subject_id,case_number)]
for i in np.arange(n_cases):
    try:
        print('Found ROIs',str(np.unique(segs[i])),'at segmentation directory file',s_directory[i],'for case',str(subject_id_corr[i]))
    except:
        print('Case',subject_id[i],'quarantined')


In [ ]:
pre_updrs_iii_off =  np.asarray(df[df.columns[3]][np.hstack((False,np.in1d(subject_id,subject_id_corr)))])                                
pre_updrs_iii_on =  np.asarray(df[df.columns[4]][np.hstack((False,np.in1d(subject_id,subject_id_corr)))])
post_updrs_iii_off =  np.asarray(df[df.columns[6]][np.hstack((False,np.in1d(subject_id,subject_id_corr)))])

per_change = (np.asarray(pre_updrs_iii_off).astype(float)-np.asarray(post_updrs_iii_off).astype(float))/(np.asarray(pre_updrs_iii_off).astype(float))
lct_change = (np.asarray(pre_updrs_iii_off).astype(float)-(np.asarray(pre_updrs_iii_on)).astype(float))/(np.asarray(pre_updrs_iii_off).astype(float))

In [ ]:
logger = logging.getLogger("radiomics")
logger.setLevel(logging.ERROR)

reextract = 0

# Assume all voxel sizes are identical
voxel_size = (0.9,0.9,0.9)
if reextract == 1:
    # Generate feature structure Phi from all ROIs and all cases
    extractor = featureextractor.RadiomicsFeatureExtractor()
    extractor.enableAllFeatures()
    extractor.enableAllImageTypes()
    extractor.enableFeatureClassByName('shape2D',enabled = False)

    seg_labels_all = np.unique(np.asarray(segs))
    Phi_gt = []
    Phi_vd = []
    Phi_lr = []
    seg_labels = []
    reextract = 0
    x_row_gt = []
    x_row_lr = []

    keylib = []
    roilib = []
    loop_count = 1
    n_rois = seg_labels_all[seg_labels_all>0].__len__()
    roi_names = np.asarray(['Background','Right substantia nigra','Right subthalamic nucleus',
                            'Left subthalamic nucleus', 'Left substantia nigra', 'Right dentate nucleus', 'Left dentate nucleus'])
    for i in np.arange(subject_id_corr.__len__()):
        seg_sitk = sitk.GetImageFromArray(segs[i])
        seg_sitk.SetSpacing(voxel_size)
        qsm_sitk_gt = sitk.GetImageFromArray(qsms[i])
        qsm_sitk_gt.SetSpacing(voxel_size)
        qsm_sitk_lr = sitk.GetImageFromArray(laros[i])
        qsm_sitk_lr.SetSpacing(voxel_size)
        # Index back since subject 12 is missing ROIs
        for j in seg_labels_all:
            if j>0:
                fv_count = 0
                featureVector_gt = extractor.execute(qsm_sitk_gt,seg_sitk,label=int(j));
                featureVector_lr = extractor.execute(qsm_sitk_lr,seg_sitk,label=int(j));
                Phi_gt.append(featureVector_gt)
                Phi_lr.append(featureVector_lr)
                for key, value in six.iteritems(featureVector_gt):
                    if 'diagnostic' in key:
                        next
                    else:
                        x_row_gt.append(featureVector_gt[key])
                        x_row_lr.append(featureVector_lr[key])
                        fv_count = fv_count+1
                        keylib.append(key)
                        roilib.append(roi_names[int(j)])
                x_row_gt.append(pre_updrs_iii_off[i])
                x_row_lr.append(pre_updrs_iii_off[i])
                fv_count = fv_count+1
        print('Extracting features for subject',subject_id_corr[i],'and appending feature matrix with vector of length',fv_count,'with UPDRS score',pre_updrs_iii_off[i])
                
    X0_gt = np.array(x_row_gt)
    X0_lr = np.array(x_row_lr)
    np.save('/home/ali/RadDBS-QSM/data/npy/rp/X0_gt_chh_rois_rp.npy',X0_gt)
    np.save('/home/ali/RadDBS-QSM/data/npy/rp/X0_lr_chh_rois_rp.npy',X0_lr)

    K = np.asarray(keylib)
    R = np.asarray(roi_names)
    np.save('/home/ali/RadDBS-QSM/data/npy/rp/K_chh_rp.npy',K)
    np.save('/home/ali/RadDBS-QSM/data/npy/rp/R_chh_rp.npy',R)

    print('Saving ground truth feature vector')
    with open('/home/ali/RadDBS-QSM/data/npy/rp/Phi_mcl_gt_roi_chh_rp', 'wb') as fp:  
        pickle.dump(Phi_gt, fp)
    
    print('Saving undersampled feature vector')
    with open('/home/ali/RadDBS-QSM/data/npy/rp/Phi_mcl_lr_roi_chh_rp', 'wb') as fp:  
        pickle.dump(Phi_lr, fp)

else:
    X0_gt = np.load('/home/ali/RadDBS-QSM/data/npy/rp/X0_gt_chh_rois_rp.npy')
    X0_lr = np.load('/home/ali/RadDBS-QSM/data/npy/rp/X0_lr_chh_rois_rp.npy')
    K = np.load('/home/ali/RadDBS-QSM/data/npy/rp/K_chh_rp.npy')
    R = np.load('/home/ali/RadDBS-QSM/data/npy/rp/R_chh_rp.npy')
    n_rois = R.shape[0]-1
    with open('/home/ali/RadDBS-QSM/data/npy/rp/Phi_mcl_gt_roi_chh_rp', "rb") as fp:  
        Phi_gt = pickle.load(fp)
    
    with open('/home/ali/RadDBS-QSM/data/npy/rp/Phi_mcl_lr_roi_chh_rp', "rb") as fp:  
        Phi_lr = pickle.load(fp)

    

In [ ]:
n_features = 1596
n_rois = 6
X_all_c = X0_gt.reshape(n_cases,n_rois,n_features)[:,0:4,:]
K_all_c = K.reshape(n_cases,n_rois,n_features-1)[:,0:4,:]
K_all_c = np.char.add(K_all_c[0,:,:].reshape(-1,1),' ')
R_all_c = np.repeat(R[1:5],n_features-1)
K_all_c = np.char.add(np.squeeze(K_all_c),np.squeeze(R_all_c))
K_all_c = np.append(K_all_c,['pre updrs']*5)
R_all_c = np.append(R_all_c,['pre updrs']*5)

In [ ]:
n_cases = len(per_change)
n_roisc = Phi_gt.__len__()/n_cases
L = int(len(X0_gt)/n_cases)
n_features = int(L/n_rois)
subsc = subject_id_corr
pre_updrs_off = pre_updrs_iii_off
results_ls_aug = np.zeros_like(per_change)

results_lgr_aug = np.zeros_like(per_change)
results_lgrp_aug = np.zeros_like(per_change)
results_ls = np.zeros_like(per_change)
results_lgr = np.zeros_like(per_change)
results_lgrp = np.zeros_like(per_change)
r = 1

In [ ]:
# Noise compensated LASSO
retrain = 1
K_nz_nc = []
K_nz_nc_lr = []
#R_nz_cd = []
if retrain == 1:

  aug = True
  K_nz = []
  c = 0

  for j in np.arange(len(subsc)):
      test_id = subsc[j]
      test_index = subsc == test_id
      train_index = subsc != test_id
      X_train = X_all_c[train_index,:,:]
      X_test = X_all_c[test_index,:,:]
      y_train = per_change[train_index]
      y_test = per_change[test_index]

      y_cat = y_train <= 0.3
      idy = np.where(y_cat==1)
      # Cross validation
                                            
      X0_ss0,scaler_ss,X_test_ss0 = util.model_scale(skp.StandardScaler(),
                                                  X_train,train_index,X_test,
                                                  test_index,pre_updrs_off,None,None,None,None,None,None,None,None,None,False,False,False)
      cvn = 5
      cv_scores = np.zeros((cvn,1))
      cv_lgr_scores = np.zeros((cvn,1))
      rs = 1
      rcfs = 1000
      (mu, sigma) = stats.norm.fit(y_train)
      kappa = stats.skew(y_train)
      print('Label distribution of:',mu,sigma,kappa)
      for jj in np.arange(2,cvn):
        # Resample to avoid stratification errors
        while np.sum(y_cat) < cvn:
          np.random.seed(rs)
          idyr = np.random.choice(np.asarray(idy).ravel())
          X0_ss0 = np.append(X0_ss0,X0_ss0[idyr,:].reshape(1,-1),axis=0)
          y_train = np.append(y_train,y_train[idyr])
          y_cat = y_train <= 0.3
          rs = rs+1
          print('Resampled to size',y_train.shape)
        if aug == True:
          y_train_n = y_train+(1.96*sigma)*np.random.normal(0,1,1)
          y_train = np.hstack((y_train,y_train_n))
          y_cat = y_train <= 0.3
          X0_ss0 = np.vstack((X0_ss0,X0_ss0))
      
      for jj in np.arange(2,cvn):
        skf_g = sms.StratifiedKFold(n_splits=jj,shuffle=True,random_state=0)
        skf_gen = skf_g.split(X0_ss0,y_cat)
        lasso = slm.LassoLarsCV(max_iter=1000,cv=jj,n_jobs=-1,normalize=False,eps=0.1)
        lgr = slm.LogisticRegressionCV(n_jobs=-1,cv=jj,class_weight=None,penalty='l1',solver='liblinear',random_state=0)
        with warnings.catch_warnings() and np.errstate(divide='ignore', invalid='ignore'):
          # Feature selection
          warnings.filterwarnings("ignore", category=ConvergenceWarning)
          sel = skf.RFECV(lasso,step=rcfs,cv=skf_gen,n_jobs=1)
          # Stratifies classifiers automatically
          sel_lr = skf.RFECV(lgr,step=rcfs,cv=jj,n_jobs=1)
          X0_ss = sel.fit_transform(X0_ss0,y_train)
          X0_ssl = sel_lr.fit_transform(X0_ss0,y_cat)
          est_ls = lasso.fit(X0_ss,y_train)
          est_lgr = lgr.fit(X0_ssl,y_cat)
          cv_scores[jj] = est_ls.score(X0_ss,y_train)
          cv_lgr_scores[jj] = est_lgr.score(X0_ssl,y_cat)
          print('LassoCV score for',jj,'is',cv_scores[jj],'from dataset of size',X0_ss.shape)
          
      with warnings.catch_warnings() and np.errstate(divide='ignore', invalid='ignore'):        
        best_cv = np.argmax(cv_scores)
        best_cv_lgr = np.argmax(cv_lgr_scores)

        # Break any ties
        if np.sum(cv_scores == best_cv) > 1:
          cv_scores_tb = np.zeros((np.sum(cv_scores == best_cv),1))
          cv_lgr_scores_tb = np.zeros((np.sum(cv_scores == best_cv_lgr),1))
          for jjj in (cv_scores == cv_scores[best_cv]):
            if jjj > 0:
              print('Breaking tie')
              skf_g = sms.StratifiedKFold(n_splits=np.arange(2,cvn)[jjj],shuffle=True,random_state=1)
              skf_gen = skf_g.split(X0_ss0,y_cat) 
              X0_ss = sel.fit_transform(X0_ss0,y_train)
              X0_ssl = sel_lr.fit_transform(X0_ss0,y_cat)
              est_ls = lasso.fit(X0_ss,y_train)
              est_lgr = lgr.fit(X0_ssl,y_cat)
              lasso = slm.LassoLarsCV(max_iter=1000,cv=np.arange(2,cvn)[jjj],n_jobs=-1,normalize=False,eps=0.1)
              lgr = slm.LogisticRegressionCV(n_jobs=-1,cv=jj,class_weight=None,penalty='l1',solver='liblinear',random_state=1)
              est_ls = lasso.fit(X0_ss,y_train)
              est_lgr = lgr.fit(X0_ssl,y_cat)
              cv_scores_tb[jjj] = est_ls.score(X0_ss,y_train)
              cv_lgr_scores_tb[jj] = est_lgr.score(sel_lr.fit_transform(X0_ss0,y_cat),y_cat)
          best_cv = np.argmax(cv_scores_tb)
          best_cv_lgr = np.argmax(cv_lgr_scores_tb)
        
        # Fit whole dataset with optimal cv
        lasso = slm.LassoLarsCV(max_iter=1000,cv=best_cv,n_jobs=-1,normalize=False,eps=0.1)
        lgr = slm.LogisticRegressionCV(n_jobs=-1,cv=best_cv_lgr,class_weight=None,penalty='l1',solver='liblinear',random_state=1)
        sel = skf.RFECV(lasso,step=rcfs,cv=best_cv)
        sel_lr = skf.RFECV(lgr,step=rcfs,cv=best_cv)
        X0_ss = sel.fit_transform(X0_ss0,y_train)
        X_test_ss = sel.transform(X_test_ss0)
        X0_ssl = sel_lr.fit_transform(X0_ss0,y_cat)
        X_test_ssl = sel_lr.transform(X_test_ss0)
        K_ss = sel.transform(K_all_c.reshape(1,-1))
        K_ss_lr = sel_lr.transform(K_all_c.reshape(1,-1))
        # R_ss = sel.transform(R_all_c.reshape(1,-1))


      # LASSO
      with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=ConvergenceWarning)
        lasso = slm.LassoLarsCV(max_iter=1000,cv=best_cv,n_jobs=-1,normalize=False,eps=0.1)
        lgr = slm.LogisticRegressionCV(n_jobs=-1,cv=best_cv_lgr,class_weight=None,penalty='l1',solver='liblinear',random_state=1)
        est_ls = lasso.fit(X0_ss,y_train)
        est_lgr = lgr.fit(X0_ssl,y_cat)
      results_lgr_aug[c] = est_lgr.predict(X_test_ssl)
      results_lgrp_aug[c] = est_lgr.predict_proba(X_test_ssl)[0][0]
      results_ls_aug[c] = est_ls.predict(X_test_ss)
      print('Lasso predicts',str(np.round(results_ls_aug[c],4)),'and logistic regression predicts',results_lgr_aug[c],
                'for case with',str(np.round(np.repeat(per_change,r)[c],2)),'and selected CV',best_cv,'and',sum(y_cat),'minority cases')
      K_nz_nc.append(np.squeeze(K_ss)[abs(est_ls.coef_)>0])
      K_nz_nc_lr.append(np.squeeze(K_ss_lr)[np.squeeze(abs(est_lgr.coef_)>0)])
      c=c+1

  np.save('results_ls_aug_d.npy',results_ls_aug)
  np.save('results_lgr_aug_d.npy',results_lgr_aug)
  np.save('results_lgrp_aug_d.npy',results_lgrp_aug)
  np.save('results_K_nz_nc.npy',K_nz_nc)
  np.save('results_K_nz_nc_lr.npy',K_nz_nc_lr)
  np.save('beta_nz_nc.npy',est_ls.coef_[abs(est_ls.coef_)>0])
  np.save('beta_nz_nc_lr.npy',est_lgr.coef_[np.squeeze(abs(est_lgr.coef_)>0)])
else:
  results_ls_aug = np.load('results_ls_aug_d.npy')
  results_lgr_aug = np.load('results_lgr_aug_d.npy')
  results_lgrp_aug = np.load('results_lgrp_aug_d.npy')
  K_nz_nc = np.load('results_K_nz_nc.npy',allow_pickle=True)

  


In [ ]:
(abs(est_lgr.coef_)>0).shape

In [ ]:
# Vanilla Lasso
Js = []
aug = False
K_nz_vl = []
K_nz_vl_lr = []
err_var = np.zeros_like(per_change)
rerror = np.zeros_like(per_change)
kappa = []
K_nz = []
E_nz = []
c = 0

for j in np.arange(len(subsc)):
    test_id = subsc[j]
    test_index = subsc == test_id
    train_index = subsc != test_id
    X_train = X_all_c[train_index,:,:]
    X_test = X_all_c[test_index,:,:]
    y_train = per_change[train_index]
    y_test = per_change[test_index]

    y_cat = y_train <= 0.3
    idy = np.where(y_cat==1)
    # Cross validation
                                          
    X0_ss0,scaler_ss,X_test_ss0 = util.model_scale(skp.StandardScaler(),
                                                X_train,train_index,X_test,
                                                test_index,pre_updrs_off,None,None,None,None,None,None,None,None,None,False,False,False)
    cvn = 5
    cv_scores = np.zeros((cvn,1))
    cv_lgr_scores = np.zeros((cvn,1))
    rs = 1
    rcfs = 1000
    (mu, sigma) = stats.norm.fit(y_train)
    kappa = stats.skew(y_train)
    print('Label distribution of:',mu,sigma,kappa)
    for jj in np.arange(2,cvn):
      # Resample to avoid stratification errors
      while np.sum(y_cat) < cvn:
        np.random.seed(rs)
        idyr = np.random.choice(np.asarray(idy).ravel())
        X0_ss0 = np.append(X0_ss0,X0_ss0[idyr,:].reshape(1,-1),axis=0)
        y_train = np.append(y_train,y_train[idyr])
        y_cat = y_train <= 0.3
        rs = rs+1
        print('Resampled to size',y_train.shape)
      if aug == True:
        y_train_n = y_train+(1.96*sigma)*np.random.normal(0,1,1)
        y_train = np.hstack((y_train,y_train_n))
        y_cat = y_train <= 0.3
        X0_ss0 = np.vstack((X0_ss0,X0_ss0))
    
    for jj in np.arange(2,cvn):
      skf_g = sms.StratifiedKFold(n_splits=jj,shuffle=True,random_state=0)
      skf_gen = skf_g.split(X0_ss0,y_cat)
      lasso = slm.LassoLarsCV(max_iter=1000,cv=jj,n_jobs=-1,normalize=False,eps=0.1)
      with warnings.catch_warnings() and np.errstate(divide='ignore', invalid='ignore'):
        warnings.filterwarnings("ignore", category=UserWarning)
        warnings.filterwarnings("ignore", category=ConvergenceWarning)
        lgr = slm.LogisticRegressionCV(n_jobs=-1,cv=jj,class_weight=None,penalty='l1',solver='liblinear',random_state=0)
        print('Found minority samples:',sum(y_cat))
        # Feature selection
        sel = skf.RFECV(lasso,step=rcfs,cv=skf_gen,n_jobs=1)
        # Stratifies classifiers automatically
        sel_lr = skf.RFECV(lgr,step=rcfs,cv=jj,n_jobs=1)
        X0_ss = sel.fit_transform(X0_ss0,y_train)
        X0_ssl = sel_lr.fit_transform(X0_ss0,y_cat)
        est_ls = lasso.fit(X0_ss,y_train)
        est_lgr = lgr.fit(X0_ssl,y_cat)
        cv_scores[jj] = est_ls.score(X0_ss,y_train)
        cv_lgr_scores[jj] = est_lgr.score(X0_ssl,y_cat)
        print('LogisticRegressionCV score for',jj,'is',cv_lgr_scores[jj],'from dataset of size',X0_ss.shape)
        
    with warnings.catch_warnings() and np.errstate(divide='ignore', invalid='ignore'):        
      best_cv = np.argmax(cv_scores)
      best_cv_lgr = np.argmax(cv_lgr_scores)

      # Break any ties
      if np.sum(cv_scores == best_cv) > 1:
        cv_scores_tb = np.zeros((np.sum(cv_scores == best_cv),1))
        cv_lgr_scores_tb = np.zeros((np.sum(cv_scores == best_cv_lgr),1))
        for jjj in (cv_scores == cv_scores[best_cv]):
          if jjj > 0:
            print('Breaking tie')
            skf_g = sms.StratifiedKFold(n_splits=np.arange(2,cvn)[jjj],shuffle=True,random_state=1)
            skf_gen = skf_g.split(X0_ss0,y_cat) 
            X0_ss = sel.fit_transform(X0_ss0,y_train)
            X0_ssl = sel_lr.fit_transform(X0_ss0,y_cat)
            est_ls = lasso.fit(X0_ss,y_train)
            est_lgr = lgr.fit(X0_ssl,y_cat)
            lasso = slm.LassoLarsCV(max_iter=1000,cv=np.arange(2,cvn)[jjj],n_jobs=-1,normalize=False,eps=0.1)
            lgr = slm.LogisticRegressionCV(n_jobs=-1,cv=jj,class_weight=None,penalty='l1',solver='liblinear',random_state=1)
            est_ls = lasso.fit(X0_ss,y_train)
            est_lgr = lgr.fit(X0_ssl,y_cat)
            cv_scores_tb[jjj] = est_ls.score(X0_ss,y_train)
            cv_lgr_scores_tb[jj] = est_lgr.score(sel_lr.fit_transform(X0_ss0,y_cat),y_cat)
        best_cv = np.argmax(cv_scores_tb)
        best_cv_lgr = np.argmax(cv_lgr_scores_tb)
      
      # Fit whole dataset with optimal cv
      lasso = slm.LassoLarsCV(max_iter=1000,cv=best_cv,n_jobs=-1,normalize=False,eps=0.1)
      lgr = slm.LogisticRegressionCV(n_jobs=-1,cv=jj,class_weight=None,penalty='l1',solver='liblinear',random_state=0)
      sel = skf.RFECV(lasso,step=rcfs,cv=best_cv)
      sel_lr = skf.RFECV(lgr,step=rcfs,cv=best_cv)
      X0_ss = sel.fit_transform(X0_ss0,y_train)
      X_test_ss = sel.transform(X_test_ss0)
      X0_ssl = sel_lr.fit_transform(X0_ss0,y_cat)
      X_test_ssl = sel_lr.transform(X_test_ss0)
      K_ss = sel.transform(K_all_c.reshape(1,-1))
      K_ss_lr = sel_lr.transform(K_all_c.reshape(1,-1))

    # LASSO
    with warnings.catch_warnings():
      warnings.filterwarnings("ignore", category=ConvergenceWarning)
      lasso = slm.LassoLarsCV(max_iter=1000,cv=best_cv,n_jobs=-1,normalize=False,eps=0.1)
      lgr = slm.LogisticRegressionCV(n_jobs=-1,cv=best_cv_lgr,class_weight=None,penalty='l1',solver='liblinear',random_state=0)
      est_ls = lasso.fit(X0_ss,y_train)
      est_lgr = lgr.fit(X0_ssl,y_cat)
    results_lgr[c] = est_lgr.predict(X_test_ssl)
    results_lgrp[c] = est_lgr.predict_proba(X_test_ssl)[0][0]
    results_ls[c] = est_ls.predict(X_test_ss)
    print('Lasso predicts',str(np.round(results_ls[c],4)),'and logistic regression predicts',results_lgr[c],
              'for case with',str(np.round(np.repeat(per_change,r)[c],2)),'and selected CV',best_cv,'and',sum(y_cat),'minority cases')
    K_nz_vl.append(np.squeeze(K_ss)[abs(est_ls.coef_)>0])
    K_nz_vl_lr.append(np.squeeze(K_ss_lr)[np.squeeze(abs(est_lgr.coef_)>0)])
    c=c+1

np.save('results_ls_d.npy',results_ls)
np.save('results_lgr_d.npy',results_lgr)
np.save('results_lgrp_d.npy',results_lgrp)
np.save('results_K_nz_vl.npy',K_nz_vl)
np.save('results_K_nz_vl_lr.npy',K_nz_vl_lr)
np.save('beta_nz_vl.npy',est_ls.coef_[abs(est_ls.coef_)>0])
np.save('beta_nz_vl_lr.npy',est_lgr.coef_[np.squeeze(abs(est_lgr.coef_)>0)])

  


In [ ]:
# SMOGN LASSO
if retrain == 1:
  results_ls_smogn = np.zeros_like(per_change)
  aug = False
  c = 0
  K_nz_smogn = []
  #R_nz_smogn = []
  for j in np.arange(c,len(subsc)):
      test_id = subsc[j]
      test_index = subsc == test_id
      train_index = subsc != test_id
      X_train = X_all_c[train_index,:,:]
      X_test = X_all_c[test_index,:,:]
      y_train = per_change[train_index]
      y_test = per_change[test_index]

      y_cat = y_train <= 0.3
      idy = np.where(y_cat==1)
      # Cross validation

    
                                            
      X0_ss0,scaler_ss,X_test_ss0 = util.model_scale(skp.StandardScaler(),
                                                  X_train,train_index,X_test,
                                                  test_index,pre_updrs_off,None,None,None,None,None,None,None,None,None,False,False,False)
      X_smogn,y_smogn,idx_kept,sscaler = util.rad_smogn(X0_ss0,y_train,np.amax(y_train),np.amin(y_train),1,0,0.05,0.02)
      X0_ss00 = X0_ss0
      X0_ss0 = np.vstack((X0_ss0,X_smogn))
      y_train = np.hstack((y_train,y_smogn))
      y_train_0 = y_train
      cvn = 5

      cv_scores = np.zeros((cvn,1))
      rs = 1
      rcfs = 1000
      (mu, sigma) = stats.norm.fit(y_train)
      kappa = stats.skew(y_train)
      print('Label distribution of:',mu,sigma,kappa)
      for jj in np.arange(2,cvn):
        # Resample to avoid stratification errors
        while np.sum(y_cat) < cvn:
          np.random.seed(rs)
          idyr = np.random.choice(np.asarray(idy).ravel())
          X0_ss0 = np.append(X0_ss0,X0_ss0[idyr,:].reshape(1,-1),axis=0)
          y_train = np.append(y_train,y_train[idyr])
          y_cat = y_train <= 0.3
          rs = rs+1
          print('Resampled to size',y_train.shape)

      for jj in np.arange(2,cvn):
        skf_g = sms.StratifiedKFold(n_splits=jj,shuffle=True,random_state=0)
        skf_gen = skf_g.split(X0_ss0,y_cat)
        lasso = slm.LassoLarsCV(max_iter=1000,cv=jj,n_jobs=-1,normalize=False,eps=0.1)
        with warnings.catch_warnings() and np.errstate(divide='ignore', invalid='ignore'):
          # Feature selection
          warnings.filterwarnings("ignore", category=ConvergenceWarning)
          sel = skf.RFECV(lasso,step=rcfs,cv=skf_gen)
          X0_sst = sel.fit_transform(X0_ss0,y_train)
          X0_ss = X0_sst
          est_ls = lasso.fit(X0_ss,y_train)
          cv_scores[jj] = est_ls.score(X0_ss,y_train)
          print('LassoCV score for',jj,'is',cv_scores[jj],'from dataset of size',X0_ss.shape)
          
      with warnings.catch_warnings() and np.errstate(divide='ignore', invalid='ignore'):        
        best_cv = np.argmax(cv_scores)
        # Break any ties
        if np.sum(cv_scores == best_cv) > 1:
          cv_scores_tb = np.zeros((np.sum(cv_scores == best_cv),1))
          for jjj in (cv_scores == cv_scores(best_cv)):
            if jjj > 0:
              skf_g = sms.StratifiedKFold(n_splits=np.arange(2,cvn)[jjj],shuffle=True,random_state=1)
              skf_gen = skf_g.split(X0_ss0,y_cat) 
              X0_sst = sel.fit_transform(X0_ss0,y_train)
              X0_ss = X0_sst
              lasso = slm.LassoLarsCV(max_iter=1000,cv=np.arange(2,cvn)[jjj],n_jobs=-1,normalize=False,eps=0.1)
              est_ls = lasso.fit(X0_ss,y_train)
              cv_scores_tb[jjj] = est_ls.score(X0_ss,y_train)
          best_cv = np.argmax(cv_scores_tb)
        
        # Fit whole dataset with optimal cv
        lasso = slm.LassoLarsCV(max_iter=1000,cv=best_cv,n_jobs=-1,normalize=False,eps=0.1)
        sel = skf.RFECV(lasso,step=rcfs,cv=best_cv)
        X0_ss = sel.fit_transform(X0_ss0,y_train)
        X_test_ss = sel.transform(X_test_ss0)
        K_ss = sel.transform(K_all_c.reshape(1,-1))
        #R_ss = sel.transform(R_all_c.reshape(1,-1))
        
      # LASSO
      with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=ConvergenceWarning)
        lasso = slm.LassoLarsCV(max_iter=1000,cv=best_cv,n_jobs=-1,normalize=False,eps=0.1)
        est_ls = lasso.fit(X0_ss,y_train)
        results_ls_smogn[c] = est_ls.predict(X_test_ss)
        if results_ls_smogn[c] < 0:
            dx, y_n = cKDTree(X0_ss00.reshape(X0_ss00.shape[0],-1)).query(X_test_ss0.reshape(1,-1),k=15)
            results_ls_smogn[c] = np.mean((y_train_0[y_n]))
            print('Using nearest neighbor')
        print('Lasso predicts',str(np.round(results_ls_smogn[c],4)),
              'for case with',str(np.round(np.repeat(per_change,r)[c],2)),'and selected CV',best_cv)
        K_nz_smogn.append(np.squeeze(K_ss)[abs(est_ls.coef_)>0])
        #R_nz_smogn.append(np.squeeze(R_ss)[est_ls.coef_>0])
        c=c+1


  np.save('results_ls_smogn_d.npy',results_ls_smogn)
  np.save('results_K_nz_smogn.npy',K_nz_smogn)
  np.save('beta_nz_smogn.npy',est_ls.coef_[abs(est_ls.coef_)>0])

else:
  results_ls_smogn = np.load('results_ls_smogn_d.npy')
  K_nz_smogn = np.load('results_K_nz_smogn.npy',allow_pickle=True)
  

  


Define probability of an unsucessful procedure as $P = 1-u$ where $u$ is the UPDRS percent improvement prediction between $0$ and $1$.

In [ ]:
# Wild bootstrap LASSO
if retrain == 1:
  aug = False
  results_ls_wbs = np.zeros_like(per_change)
  K_nz_wbs = []
  #R_nz_wbs = []
  c = 0
  for j in np.arange(c,len(subsc)):
      test_id = subsc[j]
      test_index = subsc == test_id
      train_index = subsc != test_id
      X_train = X_all_c[train_index,:,:]
      X_test = X_all_c[test_index,:,:]
      y_train = per_change[train_index]
      y_test = per_change[test_index]

      y_cat = y_train <= 0.3
      idy = np.where(y_cat==1)
      # Cross validation
                                            
      X0_ss0,scaler_ss,X_test_ss0 = util.model_scale(skp.StandardScaler(),
                                                  X_train,train_index,X_test,
                                                  test_index,pre_updrs_off,None,None,None,None,None,None,None,None,None,False,False,False)
      X0_ss00 = X0_ss0
      y_train_0 = y_train
      cvn = 5

      cv_scores = np.zeros((cvn,1))
      rs = 1
      rcfs = 1000
      (mu, sigma) = stats.norm.fit(y_train)
      kappa = stats.skew(y_train)
      print('Label distribution of:',mu,sigma,kappa)
      for jj in np.arange(2,cvn):
        # Resample to avoid stratification errors
        while np.sum(y_cat) < cvn:
          np.random.seed(rs)
          idyr = np.random.choice(np.asarray(idy).ravel())
          X0_ss0 = np.append(X0_ss0,X0_ss0[idyr,:].reshape(1,-1),axis=0)
          y_train = np.append(y_train,y_train[idyr])
          y_cat = y_train <= 0.3
          rs = rs+1
          print('Resampled to size',y_train.shape)
          ls0 = slm.LassoLarsCV(max_iter=1000,cv=jj,n_jobs=-1,normalize=False,eps=0.1)
          est0 = ls0.fit(X0_ss0,y_train)
          eps = y_train-ls0.predict(X0_ss0)
          eps_v = eps*np.random.normal(0,1,1)
          y_train_0 = y_train
        if aug == True:
          y_train_n = y_train+(1.96*sigma)*np.random.normal(0,1,1)
          y_train = np.hstack((y_train,y_train_n))
          y_cat = y_train <= 0.3
          X0_ss0 = np.vstack((X0_ss0,X0_ss0))
        else: # Control for different training sample sizes
          while len(eps_v) < len(y_train):
            eps_v = np.hstack((eps_v,eps*np.random.normal(0,1,1)))
          y_train_n = y_train+eps_v
          y_train = np.hstack((y_train,y_train_n))
          y_cat = y_train <= 0.3
          X0_ss0 = np.vstack((X0_ss0,X0_ss0))

      for jj in np.arange(2,cvn):
        skf_g = sms.StratifiedKFold(n_splits=jj,shuffle=True,random_state=0)
        skf_gen = skf_g.split(X0_ss0,y_cat)
        lasso = slm.LassoLarsCV(max_iter=1000,cv=jj,n_jobs=-1,normalize=False,eps=0.1)
        with warnings.catch_warnings() and np.errstate(divide='ignore', invalid='ignore'):
          # Feature selection
          warnings.filterwarnings("ignore", category=ConvergenceWarning)
          sel = skf.RFECV(lasso,step=rcfs,cv=skf_gen)
          X0_sst = sel.fit_transform(X0_ss0,y_train)
          X0_ss = X0_sst
          est_ls = lasso.fit(X0_ss,y_train)
          cv_scores[jj] = est_ls.score(X0_ss,y_train)
          print('LassoCV score for',jj,'is',cv_scores[jj],'from dataset of size',X0_ss.shape)
          
      with warnings.catch_warnings() and np.errstate(divide='ignore', invalid='ignore'):        
        best_cv = np.argmax(cv_scores)
        # Break any ties
        if np.sum(cv_scores == best_cv) > 1:
          cv_scores_tb = np.zeros((np.sum(cv_scores == best_cv),1))
          for jjj in (cv_scores == cv_scores(best_cv)):
            if jjj > 0:
              skf_g = sms.StratifiedKFold(n_splits=np.arange(2,cvn)[jjj],shuffle=True,random_state=1)
              skf_gen = skf_g.split(X0_ss0,y_cat) 
              X0_sst = sel.fit_transform(X0_ss0,y_train)
              X0_ss = X0_sst
              lasso = slm.LassoLarsCV(max_iter=1000,cv=np.arange(2,cvn)[jjj],n_jobs=-1,normalize=False,eps=0.1)
              est_ls = lasso.fit(X0_ss,y_train)
              cv_scores_tb[jjj] = est_ls.score(X0_ss,y_train)
          best_cv = np.argmax(cv_scores_tb)
        
        # Fit whole dataset with optimal cv
        lasso = slm.LassoLarsCV(max_iter=1000,cv=best_cv,n_jobs=-1,normalize=False,eps=0.1)
        sel = skf.RFECV(lasso,step=rcfs,cv=best_cv)
        X0_ss = sel.fit_transform(X0_ss0,y_train)
        X_test_ss = sel.transform(X_test_ss0)
        K_ss = sel.transform(K_all_c.reshape(1,-1))
        #R_ss = sel.transform(R_all_c.reshape(1,-1))
        
      # LASSO
      with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=ConvergenceWarning)
        lasso = slm.LassoLarsCV(max_iter=1000,cv=best_cv,n_jobs=-1,normalize=False,eps=0.1)
        est_ls = lasso.fit(X0_ss,y_train)
        results_ls_wbs[c] = est_ls.predict(X_test_ss)
        if results_ls_wbs[c] < 0:
            dx, y_n = cKDTree(X0_ss00.reshape(X0_ss00.shape[0],-1)).query(X_test_ss0.reshape(1,-1),k=15)
            results_ls_wbs[c] = np.mean((y_train_0[y_n]))
            print('Using nearest neighbor')
        print('Lasso predicts',str(np.round(results_ls_wbs[c],4)),
              'for case with',str(np.round(np.repeat(per_change,r)[c],2)),'and selected CV',best_cv)
        K_nz_wbs.append(np.squeeze(K_ss)[abs(est_ls.coef_)>0])
        #R_nz_wbs.append(np.squeeze(R_ss)[est_ls.coef_>0])
        c=c+1

  np.save('results_ls_wbs_d.npy',results_ls_wbs)
  np.save('results_K_nz_wbs.npy',K_nz_wbs)
  np.save('beta_nz_wbs.npy',est_ls.coef_[abs(est_ls.coef_)>0])
else:
  results_ls_wbs = np.load('results_ls_wbs_d.npy')
  K_nz_wbs = np.load('results_K_nz_wbs.npy',allow_pickle=True)

  


In [ ]:
def bland_altman_plot(data1, data2, *args, **kwargs):
    data1     = np.asarray(data1)
    data2     = np.asarray(data2)
    mean      = np.mean([data1, data2], axis=0)
    diff      = data1-data2                   # Difference between datasets
    md        = np.mean(diff)                 # Mean of the difference
    sd        = np.std(diff, axis=0)          # Standard deviation of the difference

    plt.scatter(mean, diff, *args, **kwargs)
    plt.axhline(md,           color='gray', linestyle='--')
    plt.text(0.1, 2*md, r'$\mu$ = '+str(np.round(md,2)), horizontalalignment='center',verticalalignment='center')
    plt.text(0.5, 0.4, r'$\mu + 1.96 \sigma$ = '+str(np.round(md + 1.96*sd,2)), horizontalalignment='center',verticalalignment='center')
    plt.text(0.5, -0.5, r'$\mu - 1.96 \sigma$ = '+str(np.round(md - 1.96*sd,2)), horizontalalignment='center',verticalalignment='center')
    plt.axhline(md + 1.96*sd, color='gray', linestyle='--')
    plt.axhline(md - 1.96*sd, color='gray', linestyle='--')
    plt.ylim([-4*1.96*sd,4*1.96*sd])
    diff_in = np.logical_and(diff < md + 1.96*sd,diff > md - 1.96*sd)
    plt.scatter(mean[diff_in], diff[diff_in], *args, **kwargs)
    plt.xlabel(r'$\frac{y_{true}+y_{predicted}}{2}$',fontsize=16)
    plt.ylabel(r'$y_{predicted}-y_{true}$',fontsize=12)
    return diff,diff_in

In [ ]:
d,din = bland_altman_plot(results_ls_aug,per_change)

In [ ]:
ylim = 1.25
fs = 32
ofx = 0.25
ofy = 0.75
pre_imp = np.repeat((np.asarray(pre_updrs_iii_off,dtype=float)-np.asarray(pre_updrs_iii_on,dtype=float))/np.asarray(pre_updrs_iii_off,dtype=float),r)
per_change = np.repeat(per_change,r)
plt.rcParams['figure.figsize'] = [40, 10]
fig,ax = plt.subplots(1,5,sharex=True,sharey=True)
#ax[0].scatter(pre_imp,per_change)
col = np.where(per_change <= 0.3,'orange','blue')
ax[0].scatter(pre_imp,per_change, c=col,linewidth=0)
lr = stats.linregress(pre_imp,per_change)
y_model = pre_imp*lr.slope+lr.intercept
ax[0].plot(pre_imp,y_model,color='r')
ax[0].text(ofx,ofy,'$y$ = '+str(np.round(lr.slope,2))+'$x$ + '+str(np.round(lr.intercept,2))+'\n'+'$r$ = '+str(np.round(lr.rvalue,2))+'\n'+'$p$ = '+str(('{:.3f}'.format(lr.pvalue))),
                    ha='left', va='bottom', transform=ax[0].transAxes,fontsize=fs)  
ax[0].hlines(0.3,0,2,linestyle='dashed',color='black')
ax[0].vlines(0.3,0,2,linestyle='dashed',color='black')
ax[0].set_title('LCT',fontsize=fs)
ax[0].set_xlabel('Prediction',fontsize=fs)
ax[0].xaxis.set_tick_params(labelleft=True,labelsize=fs//1.5)
ax[0].set_ylabel('True UPDRS-III improvement',fontsize=fs)
ax[0].yaxis.set_tick_params(labelleft=True,labelsize=fs//1.5)
col = np.where(per_change <= 0.3,'orange','blue')

ax[1].scatter(results_ls,per_change, c=col,linewidth=0)
lr = stats.linregress(np.squeeze(results_ls),per_change)
y_model = results_ls*lr.slope+lr.intercept
ax[1].plot(results_ls,y_model,color='r')
ax[1].text(ofx,ofy,'$y$ = '+str(np.round(lr.slope,2))+'$x$ + '+str(np.round(lr.intercept,2))+'\n'+'$r$ = '+str(np.round(lr.rvalue,2))+'\n'+'$p$ = '+str(('{:.7f}'.format(lr.pvalue))),
                    ha='left', va='bottom', transform=ax[1].transAxes,fontsize=fs) 
ax[1].hlines(0.3,0,2,linestyle='dashed',color='black')
ax[1].vlines(0.3,0,2,linestyle='dashed',color='black')
ax[1].set_title('Lasso',fontsize=fs)
ax[1].set_xlabel('Prediction',fontsize=fs)
ax[1].xaxis.set_tick_params(labelleft=True,labelsize=fs//1.5)
# ax[1].set_ylabel('True Improvement',fontsize=fs)
ax[1].yaxis.set_tick_params(labelleft=True,labelsize=fs//1.5)
col = np.where(per_change <= 0.3,'orange','blue')


ax[2].scatter(results_ls_smogn,per_change, c=col,linewidth=0)
lr = stats.linregress(np.squeeze(results_ls_smogn),per_change)
y_model = results_ls_smogn*lr.slope+lr.intercept
ax[2].plot(results_ls_smogn,y_model,color='r')
ax[2].text(ofx,ofy,'$y$ = '+str(np.round(lr.slope,2))+'$x$ + '+str(np.round(lr.intercept,2))+'\n'+'$r$ = '+str(np.round(lr.rvalue,2))+'\n'+'$p$ = '+str(('{:.7f}'.format(lr.pvalue))),
                    ha='left', va='bottom', transform=ax[2].transAxes,fontsize=fs) 
ax[2].hlines(0.3,0,2,linestyle='dashed',color='black')
ax[2].vlines(0.3,0,2,linestyle='dashed',color='black')
ax[2].set_xlabel('Prediction',fontsize=fs)
ax[2].xaxis.set_tick_params(labelleft=True,labelsize=fs//1.5)
# ax[2].set_ylabel('True Improvement',fontsize=fs)
ax[2].yaxis.set_tick_params(labelleft=True,labelsize=fs//1.5)
ax[2].set_title('SMOGN Lasso',fontsize=fs)

ax[3].scatter(results_ls_wbs,per_change, c=col,linewidth=0)
lr = stats.linregress(np.squeeze(results_ls_wbs),per_change)
y_model = results_ls_wbs*lr.slope+lr.intercept
ax[3].plot(results_ls_wbs,y_model,color='r')
ax[3].text(ofx,ofy,'$y$ = '+str(np.round(lr.slope,2))+'$x$ + '+str(np.round(lr.intercept,2))+'\n'+'$r$ = '+str(np.round(lr.rvalue,2))+'\n'+'$p$ = '+str(('{:.7f}'.format(lr.pvalue))),
                    ha='left', va='bottom', transform=ax[3].transAxes,fontsize=fs) 
ax[3].hlines(0.3,0,2,linestyle='dashed',color='black')
ax[3].vlines(0.3,0,2,linestyle='dashed',color='black')
ax[3].set_xlabel('Prediction',fontsize=fs)
ax[3].xaxis.set_tick_params(labelleft=True,labelsize=fs//1.5)
# ax[3].set_ylabel('True Improvement',fontsize=fs)
ax[3].yaxis.set_tick_params(labelleft=True,labelsize=fs//1.5)
ax[3].set_title('Wild Boostrap Lasso',fontsize=fs)

ax[4].scatter(results_ls_aug,per_change, c=col,linewidth=0)
lr = stats.linregress(np.squeeze(results_ls_aug),per_change)
y_model = results_ls_aug*lr.slope+lr.intercept
ax[4].plot(results_ls_aug,y_model,color='r')
ax[4].text(ofx,ofy,'$y$ = '+str(np.round(lr.slope,2))+'$x$ + '+str(np.round(lr.intercept,2))+'\n'+'$r$ = '+str(np.round(lr.rvalue,2))+'\n'+'$p$ = '+str(('{:.7f}'.format(lr.pvalue))),
                    ha='left', va='bottom', transform=ax[4].transAxes,fontsize=fs) 
ax[4].hlines(0.3,0,2,linestyle='dashed',color='black')
ax[4].vlines(0.3,0,2,linestyle='dashed',color='black')
ax[4].set_xlabel('Prediction',fontsize=fs)
ax[4].xaxis.set_tick_params(labelleft=True,labelsize=fs//1.5)
# ax[4].set_ylabel('True Improvement',fontsize=fs)
ax[4].yaxis.set_tick_params(labelleft=True,labelsize=fs//1.5)
ax[4].set_title('Noise Compensated Lasso',fontsize=fs)
plt.ylim([0,ylim])
plt.xlim([0,ylim])



In [ ]:
from collections import Counter

plt.rcParams["figure.figsize"] = (30,15)
fig, axes = plt.subplots(1,4,sharey=True)
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.2, 
                    hspace=0.2)

R = [item for sublist in K_nz_vl for item in sublist]
letter_counts = Counter(R)
lc = { x: count for x, count in letter_counts.items() if count > 30 }
df = pd.DataFrame.from_dict(lc, orient='index')
df.sort_values(0, ascending=False, inplace=True)
df.plot(ax=axes[0], y=0, kind='bar', legend=False, fontsize=fs,stacked=True, width=1, edgecolor='black',color='tab:orange')
axes[0].set_title('Lasso',fontsize=fs)
R = [item for sublist in K_nz_smogn for item in sublist]
letter_counts = Counter(R)
lc = { x: count for x, count in letter_counts.items() if count > 23 }
df = pd.DataFrame.from_dict(lc, orient='index')
df.sort_values(0, ascending=False, inplace=True)
df.plot(ax=axes[1], y=0, kind='bar', legend=False, fontsize=fs,stacked=True, width=1, edgecolor='black',color='tab:red')
axes[1].set_title('SMOGN Lasso',fontsize=fs)
R = [item for sublist in K_nz_wbs for item in sublist]
letter_counts = Counter(R)
lc = { x: count for x, count in letter_counts.items() if count > 28 }
df = pd.DataFrame.from_dict(lc, orient='index')
df.sort_values(0, ascending=False, inplace=True)
df.plot(ax=axes[2], y=0, kind='bar', legend=False, fontsize=fs,stacked=True, width=1, edgecolor='black',color='tab:purple')
axes[2].set_title('Wild Bootstrap Lasso',fontsize=fs)
R = [item for sublist in K_nz_nc for item in sublist]
letter_counts = Counter(R)
lc = { x: count for x, count in letter_counts.items() if count > 25 }
df = pd.DataFrame.from_dict(lc, orient='index')
df.sort_values(0, ascending=False, inplace=True)
df.plot(ax=axes[3], y=0, kind='bar', legend=False, fontsize=fs,stacked=True, width=1, edgecolor='black',color='tab:green')
axes[3].set_title('Noise Compensated Lasso',fontsize=fs)
plt.suptitle('Predictive radiomic features',fontsize=fs)
plt.ylabel('Frequency',fontsize=16)

plt.style.use('default')

In [ ]:
plt.rcParams["figure.figsize"] = (30,15)
fig,ax = plt.subplots(1,2,sharex=True,sharey=True)
t = 0.3
y_bin = per_change<t

y_predicted_ls = 1-results_ls
y_predicted_ls_aug = 1-results_ls_aug
y_predicted_lct = 1-pre_imp
y_predicted_aug = 1-results_lgrp_aug
y_predicted_lr = 1-results_lgrp
y_predicted_smogn = 1-results_ls_smogn
y_predicted_wbs = 1-results_ls_wbs

fprlsa, tprlsa, _ = roc_curve(y_bin,  y_predicted_ls_aug)
fprls, tprls, _ = roc_curve(y_bin,  y_predicted_ls)
fprl, tprl, _ = roc_curve(y_bin,  y_predicted_lct)
fpra, tpra, _ = roc_curve(y_bin, y_predicted_aug)
fprlr, tprlr, _ = roc_curve(y_bin, y_predicted_lr)
fprlss, tprlss, _ = roc_curve(y_bin, y_predicted_smogn)
fprlwb, tprlwb, _ = roc_curve(y_bin, y_predicted_wbs)

roc_auc_lct = auc(fprl, tprl)
roc_auc_lr = auc(fprlr, tprlr)
roc_auc_lr_aug = auc(fpra, tpra)
roc_auc_ls = auc(fprls, tprls)
roc_auc_ls_aug = auc(fprlsa, tprlsa)
roc_auc_ls_smogn = auc(fprlss, tprlss)
roc_auc_ls_wbs = auc(fprlwb, tprlwb)

plt.rcParams['legend.loc']='lower right'
ax[0].plot(fprl, tprl, label = 'LCT AUC = %0.2f' % roc_auc_lct, linewidth=5)
ax[0].plot(fprlr, tprlr, label = 'LR AUC = %0.2f' % roc_auc_lr, linewidth=5)
ax[0].plot(fpra, tpra, label = 'LR + noise AUC = %0.2f' % roc_auc_lr_aug, linewidth=5)
# plt.plot(fprls, tprls, label = 'LASSO AUC = %0.2f' % roc_auc_ls)
# plt.plot(fprlsa, tprlsa, label = 'LASSO + noise AUC = %0.2f' % roc_auc_ls_aug)
ax[0].set_title('Classifier performance',fontsize=fs)
ax[0].set_xlabel('False positive rate \n $(1-specificity$)',fontsize=fs)
ax[0].set_ylabel('True positive rate \n $(sensitivity$)',fontsize=fs)
ax[0].xaxis.set_tick_params(labelleft=True,labelsize=fs//1.5)
ax[0].yaxis.set_tick_params(labelleft=True,labelsize=fs//1.5)
ax[0].legend(fontsize=fs//1.5)
plt.rcParams['legend.loc']='lower right'
ax[1].plot(fprl, tprl, label = 'LCT AUC = %0.2f' % roc_auc_lct, linewidth=5)
ax[1].plot(fprls, tprls, label = 'LASSO AUC = %0.2f' % roc_auc_ls, linewidth=5)
ax[1].plot(fprlsa, tprlsa, label = 'LASSO + noise AUC = %0.2f' % roc_auc_ls_aug, linewidth=5)
ax[1].plot(fprlss, tprlss, label = 'LASSO + SMOGN AUC = %0.2f' % roc_auc_ls_smogn, linewidth=5)
ax[1].plot(fprlwb, tprlwb, label = 'LASSO + wild bootstrap AUC = %0.2f' % roc_auc_ls_wbs, linewidth=5)
# plt.plot(fprls, tprls, label = 'LASSO AUC = %0.2f' % roc_auc_ls)
# plt.plot(fprlsa, tprlsa, label = 'LASSO + noise AUC = %0.2f' % roc_auc_ls_aug)
ax[1].set_title('Binarized regressor performance',fontsize=fs)
ax[1].set_xlabel('False positive rate \n $(1-specificity$)',fontsize=fs)
ax[1].set_ylabel('True positive rate \n $(sensitivity$)',fontsize=fs)
ax[1].xaxis.set_tick_params(labelleft=True,labelsize=fs//1.5)
ax[1].yaxis.set_tick_params(labelleft=True,labelsize=fs//1.5)
ax[1].legend(fontsize=fs//1.5)
plt.style.use('default')
plt.show()

In [ ]:
results_lgrp_aug